# Módulo 2️⃣ - Validação

In [17]:
# Importação de bibliotecas
## Processamento de dados
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

## Matplotlib e estilos para criação de gráficos
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D

In [18]:
# Iremos trabalhar com dados de Pokémon (Extraídos da PokéAPI)
data = {
    'Nome': ['Charmander', 'Squirtle', 'Growlithe', 'Psyduck', 'Vulpix', 'Poliwag',
             'Magmar', 'Tentacool', 'Cyndaquil', 'Totodile', 'Torchic', 'Mudkip',
             'Chimchar', 'Piplup', 'Fennekin', 'Froakie'],
    'Tipo': ['Fogo', 'Água', 'Fogo', 'Água', 'Fogo', 'Água',
             'Fogo', 'Água', 'Fogo', 'Água', 'Fogo', 'Água',
             'Fogo', 'Água', 'Fogo', 'Água'],
    'Attack': [52, 48, 70, 52, 50, 50, 95, 40, 49, 65, 60, 70, 58, 51, 61, 56],
    'Speed': [65, 43, 60, 55, 65, 90, 93, 70, 65, 43, 45, 40, 61, 50, 72, 71]
}

In [19]:
# Criando dataframe
df_pokemon = pd.DataFrame(data)
print(df_pokemon)

          Nome  Tipo  Attack  Speed
0   Charmander  Fogo      52     65
1     Squirtle  Água      48     43
2    Growlithe  Fogo      70     60
3      Psyduck  Água      52     55
4       Vulpix  Fogo      50     65
5      Poliwag  Água      50     90
6       Magmar  Fogo      95     93
7    Tentacool  Água      40     70
8    Cyndaquil  Fogo      49     65
9     Totodile  Água      65     43
10     Torchic  Fogo      60     45
11      Mudkip  Água      70     40
12    Chimchar  Fogo      58     61
13      Piplup  Água      51     50
14    Fennekin  Fogo      61     72
15     Froakie  Água      56     71


In [20]:
# Transformando valores categóricos em números inteiros (isso permite a categorização dos dados pelo computador)
le = LabelEncoder()
df_pokemon['Tipo'] = le.fit_transform(df_pokemon['Tipo'])
print(df_pokemon)

          Nome  Tipo  Attack  Speed
0   Charmander     0      52     65
1     Squirtle     1      48     43
2    Growlithe     0      70     60
3      Psyduck     1      52     55
4       Vulpix     0      50     65
5      Poliwag     1      50     90
6       Magmar     0      95     93
7    Tentacool     1      40     70
8    Cyndaquil     0      49     65
9     Totodile     1      65     43
10     Torchic     0      60     45
11      Mudkip     1      70     40
12    Chimchar     0      58     61
13      Piplup     1      51     50
14    Fennekin     0      61     72
15     Froakie     1      56     71


In [ ]:
# Separando dados de treinamento e teste
x_pokemon = df_pokemon[['Attack', 'Speed']].values
y_pokemon = df_pokemon['Tipo'].values